# Coupang

In [2]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

driver = webdriver.Chrome()

#Input URL
url = "https://www.coupang.com/vp/products/7566062441?vendorItemId=87024065100"
driver.get(url)

#js load 될 떄 까지 delay
script = """return document.readyState === 'complete';"""
while not driver.execute_script(script):
    pass

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

try:
    #product name
    product_name = soup.find_all('h1', class_='prod-buy-header__title')
    product_price = soup.find_all('span', class_='total-price')
    
    #get option each type
    option_type1_name = soup.find_all('div', class_='option-table-list__option-name')
    option_type1_price = soup.find_all('div', class_='option-table-list__option-price')

    option_type2_name = soup.find_all('div', class_='prod-option__dropdown-item-title')
    option_type2_price = soup.find_all('div', class_="prod-option__dropdown-item-price")

    option_type3_name = soup.find_all('li', class_='Dropdown-Select__Dropdown__Item')
    option_type3_type = soup.find_all('span', class_='Text-Select__Item__Text')

except Exception as e:
    print(f"Error: {e}")

print("Product Name :")
for i in range(len(product_name)):
    print(f"{product_name[i].get_text(strip=True)} {product_price[i].get_text(strip=True)}")

print("Option List :")

TYPE = 0
if(len(option_type1_name) > 0):
    TYPE = 1
elif(len(option_type2_name) > 0):
    TYPE = 2
elif(len(option_type3_name) > 0):
    TYPE = 3

if(TYPE == 1):
    for i in range(len(option_type1_name)):
        print(f"{option_type1_name[i].get_text(strip=True)} {option_type1_price[i].get_text(strip=True)}")
elif(TYPE == 2):
    for i in range(len(option_type2_name)):
        print(f"{option_type2_name[i].get_text(strip=True)} {option_type2_price[i].get_text(strip=True)}")
elif(TYPE == 3):
    for i in range(len(option_type3_name)):
        print(f"{option_type3_name[i].get_text(strip=True)}")
    for i in range(len(option_type3_type)):
        print(f"{option_type3_type[i].get_text(strip=True)}")

driver.quit()

Product Name :
만성수산 고창 민물장어, 1kg 600g 내..., 1개 37,900원
Option List :
1kg 600g 내외 × 1개 37,900원
2kg 1200g 내외 × 1개 74,300원즉시할인가
3kg 1800g 내외 × 1개 111,700원즉시할인가


# Gmarket (옵션 구현 미흡)

In [12]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
import json

driver = webdriver.Chrome()

#Input URL
url = "https://item.gmarket.co.kr/Item?goodscode=2720975391"
driver.get(url)

#js load 될 떄 까지 delay
script = """return document.readyState === 'complete';"""
while not driver.execute_script(script):
    pass

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

try:
    #product name
    product_title = soup.find('title')

    #product options
    product_name = soup.find_all('span', class_='item_tit')

    #options js
    scripts = soup.find_all('script')

except Exception as e:
    print(f"Error: {e}")

print("Product Name :")
print(f"{product_title.get_text(strip=True)}")

print("Product Option :")
#options
for i in range(len(product_name)):
    print(f"{product_name[i].get_text(strip=True)}")


for script in scripts:
    if script.string:
        #split crawling js to 1 line
        lines = script.string.splitlines()
        for line in lines:
            #option이 들어가있는 내용이면, 가져옴
            if "GmktItem.OptionParamCoreAbove.combOptionObj" in line:
                js = line
                break

#crawling result -> json
#js_result = re.findall(r"Request\.ItemRequests\s*=\s*(\[\[.*?\]\]);", js, re.DOTALL)

js_result = re.findall(r"GmktItem\.OptionParamCoreAbove\.combOptionObj\s*=\s*(\{.*?\}(?:,\s*\{.*?\})*);", js, re.DOTALL)
print(js_result)
option_json = js_result[0]

try:
    data = json.loads(option_json)
except json.JSONDecodeError as e:
    print("Error decoding JSON:", e)
    data = None

option_list = {}
if data:
    for outer_list in data:
        for item in outer_list:
            if 'OrderSectionName' in item:
                
                option_1 = item['OrderSectionName']
                option_2 = item['OrderText']
                option_3 = item['OrderText2']
                option_4 = item['OrderText']

                if(option_1 not in option_list):
                    option_list[option_1] = []
                else:
                    option_list[option_1].append((option_2, option_3, option_4))

for key, items in option_list.items():
    print(f"{key}:")
    for item in items:
        print(f"  - {item[0]}")
    print()

driver.quit()

# Auction 99% 완

In [11]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
import json

driver = webdriver.Chrome()

#Input URL
url = "http://itempage3.auction.co.kr/DetailView.aspx?ItemNo=E410585427"
driver.get(url)

#js load 될 떄 까지 delay
script = """return document.readyState === 'complete';"""
while not driver.execute_script(script):
    pass

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

try:
    #product name
    product_title = soup.find('title')
    product_name = soup.find_all('span', class_='item_tit')
    product_price = soup.find('span', class_ ="item_price")

    #option
    option_type1_name = soup.find_all('span', class_ = "option_prod")
    
    scripts = soup.find_all('script')

except Exception as e:
    print(f"Error: {e}")

print("Product Name :")
print(f"{product_title.get_text(strip=True)}")

print("Product Option :")
#options
for i in range(1, len(option_type1_name) // 2):
    print(f"{option_type1_name[i].get_text(strip=True)}")

for script in scripts:
    if script.string:
        #split crawling js to 1 line
        lines = script.string.splitlines()
        for line in lines:
            #option이 들어가있는 내용이면, 가져옴
            if "Request.ItemRequests=" in line:
                js = line
                break

#crawling result -> json
js_result = re.findall(r"Request\.ItemRequests\s*=\s*(\[\[.*?\]\]);", js, re.DOTALL)
if(len(js_result) > 0):
    option_json = js_result[0]

    try:
        data = json.loads(option_json)
    except json.JSONDecodeError as e:
        print("Error decoding JSON:", e)
        data = None

    option_list = {}
    if data:
        for outer_list in data:
            for item in outer_list:
                if 'OrderSectionName' in item:
                    
                    option_1 = item['OrderSectionName']
                    option_2 = item['OrderText']
                    option_3 = item['OrderText2']
                    option_4 = item['OrderText']

                    if(option_1 not in option_list):
                        option_list[option_1] = []
                    else:
                        option_list[option_1].append((option_2, option_3, option_4))

    for key, items in option_list.items():
        print(f"{key}:")
        for item in items:
            print(f"  - {item[0]}")
        print()
else:
    print("This product has no option.")

driver.quit()

Product Name :
빠른배송 헤드앤숄더 클리니컬 스트렝스 저자극 비듬 샴푸 400ml 2개 / 도움 - 옥션
Product Option :
This product has no option.


# 11ST

In [25]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
import time
import json

driver = webdriver.Chrome()

#Input URL
url = "https://www.11st.co.kr/products/2416794313?&trTypeCd=PW00&trCtgrNo=585021&checkCtlgPrd=true"
driver.get(url)

#js load 될 떄 까지 delay
script = """return document.readyState === 'complete';"""
while not driver.execute_script(script):
    pass


html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

try:
    #product name
    product_title = soup.find('title')

    for div_tag in soup.find_all('div', class_='option_item_info cont'):
        strong_tags = div_tag.find_all('strong') 
        for strong_tag in strong_tags:
            print(strong_tag.text.strip())  

    scripts = soup.find_all('script')

except Exception as e:
    print(f"Error: {e}")

print("Product Name :")
print(f"{product_title.get_text(strip=True)}")


for script in scripts:
    if script.string:
        #split crawling js to 1 line
        lines = script.string.splitlines()
        for line in lines:
            print(line)
            print()

driver.quit()

<html lang="ko" class="js-focus-visible" data-js-focus-visible=""><head><script type="text/javascript" async="" src="https://www.googletagmanager.com/gtag/js?id=G-6VBF5N51X2&amp;l=dataLayer&amp;cx=c"></script>
        <title>[95-140] 여름 남자 반팔 셔츠/빅사이즈/와이셔츠/린넨/남방/체크/오버핏/7부/스트라이프/하와이안/큰옷</title>
        <meta name="title" content="[11번가] [95-140] 여름 남자 반팔 셔츠/빅사이즈/와이셔츠/린넨/남방/체크/오버핏/7부/스트라이프/하와이안/큰옷">
        <meta name="description" content="[95-140] 여름 남자 반팔 셔츠/빅사이즈/와이셔츠/린넨/남방/체크/오버핏/7부/스트라이프/하와이안/큰옷, 할인모음가: 13,900원, 평점: 4.5, 리뷰수: 1029">
        <meta name="keyword" content="남성의류>셔츠/남방>스트라이프 셔츠/남방 ">
        <link rel="alternate" media="only screen and (mix-width: 640px)" href="https://m.11st.co.kr/products/2416794313">
        <link rel="canonical" href="https://www.11st.co.kr/products/2416794313">

        <meta http-equiv="Content-Type" content="text/html; charset=UTF-8">
        <meta http-equiv="X-UA-Compatible" content="IE=edge">
        <meta property="og:type" content="website">
 